# EXPLORATION_14 아이유팬이 좋아할만한 다른 아티스트 찾기

- 추천 시스템이란?   
=> 나와 비슷한 다른 사용자들이 좋아하는 것과 비슷한 것을 내게 추쳔해주는 것!   

- 협업 필터링방식과 콘텐츠 기반 필터링방식의 차이점은?  
    - 협업 필터링은 다수의 사용자의 아이템 구매 이력 정보만으로 사용자간 유사성 및 아이템 간 유사성을 파악함  
    - 콘텐츠 기반 필터링은 아이템의 고유의 정보를 바탕으로 아이템 간 유사성을 파악한다.

=> 콘텐츠 기반 필터링에서는 아이템 자체의 속성에만 주목하고 사용자와 아이템 간의 관련성 에는 주목하지 않는다.

추천시스템은  
1. 아이템이 많고,   
2. 유저의 취향은 다양할 때 유저가 소비할 만한 아이템을 예측하는 모델이다.   
ex) 유튜브, 페이스북, 아마존 등이 있다.

## 1) 데이터 준비와 전처리

In [1]:
## 필요한 라이브러리 import하기!

import os
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

print("뿅~")

뿅~


In [2]:
## 저장되어있는 데이터를 불러오는데 위에서 5개만 ! 확인해보기!

rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'ratings', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,ratings,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [3]:
## 평점 3점 이상만 남기고 3점 미만은 다 제외시키고 보겠다~~~~

ratings = ratings[ratings['ratings']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [4]:
## ratings 컬럼의 이름을 counts로 바꿔준다.

ratings.rename(columns={'ratings':'counts'}, inplace=True)

In [5]:
## 컬럼이름이 잘 되었는지 확인해보기!

ratings['counts']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: counts, Length: 836478, dtype: int64

In [6]:
## 영화 제목을 보기 위해 메타 데이터를 불러오기!

movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


## 2) 분석해 봅시다.

- ratings 에 있는 유니크한 영화 개수  
- ratings 에 있는 유니크한 사용자 수  
- 가장 인기 있는 영화 30개(인기순)  

** pandas.DataFrame.nunique() 은 특정 컬럼에 포함된 유니크한 데이터의 개수를 알아보는데 유용함 **

In [7]:
##ratings에 있는 유니크한 영화 개수

print(ratings['movie_id'].nunique())  

3628


In [8]:
## rating에 있는 유니크한 사용자 수

print(ratings['user_id'].nunique())  

6039


In [9]:
## 가장 인기 있는 영화 30개(인기순)

mv_count = ratings.groupby('movie_id')['user_id'].count()
mv_count.sort_values(ascending=False).head(30)

movie_id
2858    3211
260     2910
1196    2885
1210    2716
2028    2561
589     2509
593     2498
1198    2473
1270    2460
2571    2434
480     2413
2762    2385
608     2371
110     2314
1580    2297
527     2257
1197    2252
2396    2213
1617    2210
318     2194
858     2167
1265    2121
1097    2102
2997    2066
2716    2051
296     2030
356     2022
1240    2019
1       2000
457     1941
Name: user_id, dtype: int64

In [10]:
## 앞선 두 데이터프레임을 합쳐서 출력해보기


mv_total = pd.merge(ratings, movies, how='left')
mv_total

mv_count = mv_total.groupby('title')['user_id'].count()
mv_count.sort_values(ascending=False).head(30)

title
American Beauty (1999)                                   3211
Star Wars: Episode IV - A New Hope (1977)                2910
Star Wars: Episode V - The Empire Strikes Back (1980)    2885
Star Wars: Episode VI - Return of the Jedi (1983)        2716
Saving Private Ryan (1998)                               2561
Terminator 2: Judgment Day (1991)                        2509
Silence of the Lambs, The (1991)                         2498
Raiders of the Lost Ark (1981)                           2473
Back to the Future (1985)                                2460
Matrix, The (1999)                                       2434
Jurassic Park (1993)                                     2413
Sixth Sense, The (1999)                                  2385
Fargo (1996)                                             2371
Braveheart (1995)                                        2314
Men in Black (1997)                                      2297
Schindler's List (1993)                                  2257
Pr

## 3) 내가 선호하는 영화를 5가지 골라서 ratings에 추가해 줍시다.

In [11]:
## 일단 영화제목을 찾아보자! (내가 원하는 영화가 있을 수도 있고 없을 수도 있으니까!!!!!)
## 1. toy story

movies[movies['title'].str.lower().str.contains('toy story')]

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
3045,3114,Toy Story 2 (1999),Animation|Children's|Comedy


In [12]:
## 2. men in black

movies[movies['title'].str.lower().str.contains('men in black')]

,movie_id,title,genre
1539,1580,Men in Black (1997),Action|Adventure|Comedy|Sci-Fi


In [13]:
## 3. titanic

movies[movies['title'].str.lower().str.contains('titanic')]

,movie_id,title,genre
1672,1721,Titanic (1997),Drama|Romance
2088,2157,"Chambermaid on the Titanic, The (1998)",Romance
3334,3403,Raise the Titanic (1980),Drama|Thriller
3335,3404,Titanic (1953),Action|Drama


In [14]:
## 사용자 인덱스 생성

my_id = ratings['user_id'].max() + 1
print(my_id)

6041


In [15]:
ratings.head(10)

,user_id,movie_id,counts,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
5,1,1197,3,978302268
6,1,1287,5,978302039
7,1,2804,5,978300719
8,1,594,4,978302268
9,1,919,4,978301368


In [16]:
## 위에 결과를 보니 필요없는 칼럼인 timestamp 가 있어서 제거해주자!

using_cols = ['user_id', 'movie_id', 'counts']
ratings = ratings[using_cols]
ratings

,user_id,movie_id,counts
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5
...,...,...,...
1000203,6040,1090,3
1000205,6040,1094,5
1000206,6040,562,5
1000207,6040,1096,4


In [17]:
ratings.tail(10) 

,user_id,movie_id,counts
1000198,6040,2021,3
1000199,6040,2022,5
1000200,6040,2028,5
1000201,6040,1080,4
1000202,6040,1089,4
1000203,6040,1090,3
1000205,6040,1094,5
1000206,6040,562,5
1000207,6040,1096,4
1000208,6040,1097,4


In [18]:
## 내가 좋아하는 영화 데이터 확인

my_mvid = [1, 3114, 1580, 1721]

my_movie_list = pd.DataFrame(
    {'user_id': [my_id] * 4, 'movie_id': my_mvid, 'counts': [5] * 4})

if not ratings.isin({'user_id': [my_id]})['user_id'].any():
    ratings = ratings.append(my_movie_list)
    

ratings.tail(10) 

,user_id,movie_id,counts
1000202,6040,1089,4
1000203,6040,1090,3
1000205,6040,1094,5
1000206,6040,562,5
1000207,6040,1096,4
1000208,6040,1097,4
0,6041,1,5
1,6041,3114,5
2,6041,1580,5
3,6041,1721,5


In [19]:
## user_id 랑 movie_id 를 고유한 값으로 만들기!

user_unique = ratings['user_id'].unique()
movie_unique = ratings['movie_id'].unique()

user_to_idx = {v:k for k,v in enumerate(user_unique)}
movie_to_idx = {v:k for k,v in enumerate(movie_unique)}

temp_user_data = ratings['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(ratings):   
    print('user_id column indexing OK!!')
    ratings['user_id'] = temp_user_data    
else:
    print('user_id column indexing Fail!!')

temp_movie_data = ratings['movie_id'].map(movie_to_idx.get).dropna()
if len(temp_movie_data) == len(ratings):
    print('movie column indexing OK!!')
    ratings['movie_id'] = temp_movie_data
else:
    print('movie column indexing Fail!!')

ratings

user_id column indexing OK!!
movie column indexing OK!!


,user_id,movie_id,counts
0,0,0,5
1,0,1,3
2,0,2,3
3,0,3,4
4,0,4,5
...,...,...,...
1000208,6038,26,4
0,6039,40,5
1,6039,50,5
2,6039,175,5


## 4) CSR matrix를 직접 만들어 봅시다.

In [20]:
n_user = ratings['user_id'].max()
n_movie = ratings['movie_id'].max()
print(n_user, n_movie)     

csr_data = csr_matrix((ratings['counts'], (ratings.user_id, ratings.movie_id)))
csr_data

6039 3627


<6040x3628 sparse matrix of type '<class 'numpy.int64'>'
	with 836482 stored elements in Compressed Sparse Row format>

## 5) als_model = AlternatingLeastSquares 모델을 직접 구성하여 훈련시켜 봅시다.

In [21]:
import implicit
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

## implicit 라이브러리에서 권장하고 있는 부분이다. 학습 내용과는 무관함
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [22]:
## Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=128, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

In [23]:
## als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해준다.
csr_data_transpose = csr_data.T
csr_data_transpose

<3628x6040 sparse matrix of type '<class 'numpy.int64'>'
	with 836482 stored elements in Compressed Sparse Column format>

In [24]:
csr_data_transpose.shape

(3628, 6040)

In [25]:
## 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/15 [00:00<?, ?it/s]

## 6) 내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악해 보세요.

In [26]:
my_mvid = [1, 3114, 1580, 1721]

In [27]:
## movie_id로 영화 title가져오기
def get_movie_name(idx):
    if idx in movies.movie_id:
        return movies[movies['movie_id'] == idx]['title'].values[0]
    else:
        print('해당 인덱스의 영화가 존재하지 않습니다.')

In [28]:
get_movie_name(1)

'Toy Story (1995)'

In [29]:
get_movie_name(3114)

'Toy Story 2 (1999)'

In [30]:
get_movie_name(1580)

'Men in Black (1997)'

In [31]:
get_movie_name(1721)

'Titanic (1997)'

In [32]:
## my_id의 벡터
mulder = user_to_idx[my_id]
my_vector = als_model.user_factors[mulder]
my_vector

array([ 0.32568532,  0.1372878 , -0.6114649 , -0.33358303, -0.11206581,
       -0.00260127,  0.24084835, -0.6683578 , -0.01239502, -0.05367496,
       -0.27770644, -0.19265881, -0.07262883,  0.10513332,  0.13155515,
       -0.6881761 , -0.06573337, -0.2219855 ,  0.5400352 , -0.6424611 ,
       -0.50373733, -0.25565976,  0.51581144, -0.39567113, -0.39019623,
        0.11767682, -0.43386835,  0.29959634, -0.02578807, -0.27080292,
       -0.50630504,  0.57533586,  0.23932552, -0.50958145,  0.42794383,
        1.0677775 ,  0.9559571 ,  0.92383045, -0.38921735, -0.43135467,
       -0.15997711,  0.3766638 ,  0.2781313 ,  0.6754183 , -0.35795152,
       -0.6292305 , -0.19331144,  0.12243497,  0.19547227, -0.19698825,
        0.4226509 ,  0.22618473,  0.4908188 , -0.07686332,  1.220215  ,
       -0.35676593, -0.26391476,  0.01689165, -0.00197709,  0.17157264,
        0.05500438, -0.29053265, -0.42318276, -0.78155786,  0.25536463,
        0.11051087,  0.39379966, -0.03872568,  0.17013735,  0.10

In [33]:
## 'Toy Story (1995)'의 벡터
toy_story = movie_to_idx[260]
toy_story_vector = als_model.item_factors[toy_story]
toy_story_vector

array([-0.01917237,  0.01923865,  0.0044138 , -0.00098112, -0.00325116,
       -0.01115048, -0.01826386, -0.02292   , -0.01157439, -0.02231839,
        0.02259148, -0.01249139,  0.00959122, -0.00138791,  0.02035173,
       -0.01558151, -0.01860175, -0.01625199, -0.00778716,  0.03145387,
        0.02477975,  0.03467689, -0.01073203, -0.00810319,  0.02104464,
       -0.01199242,  0.02563928,  0.02419729,  0.01370389,  0.00852477,
        0.00241979,  0.00433293,  0.01989476,  0.03250746,  0.01743457,
        0.01100382, -0.0004028 ,  0.03191261,  0.00833113, -0.01586457,
        0.00515113,  0.00817534, -0.00608768,  0.0123464 , -0.01862442,
        0.01493935,  0.01330989, -0.01551812,  0.01911702,  0.00958668,
        0.01888282,  0.03324556,  0.0343282 , -0.01093562,  0.01051   ,
       -0.0125359 , -0.00986101,  0.01132499,  0.01011916, -0.01962673,
        0.00022007,  0.00728764,  0.02239028, -0.02447522,  0.03705081,
        0.0115905 , -0.03947629,  0.03745819, -0.01010483,  0.01

In [34]:
np.dot(my_vector, toy_story_vector)

0.105096444

In [35]:
## 'Men in Black (1997)'  의 벡터
men_in_black = movie_to_idx[3114]
men_in_black_vector = als_model.item_factors[men_in_black]
men_in_black_vector

array([ 0.01595385,  0.00181148,  0.02676142, -0.03652593, -0.00101577,
       -0.00418329,  0.0518409 , -0.02597741, -0.01165512, -0.01440343,
       -0.02680926, -0.02375953,  0.01530027,  0.00452498,  0.04534106,
       -0.01399732,  0.01042453,  0.01294132,  0.01294549, -0.01195688,
       -0.03030765, -0.00206182,  0.03640049, -0.00961219,  0.00270708,
        0.02768481, -0.00620191,  0.00077469, -0.00368743,  0.00469424,
       -0.0286862 ,  0.04599993, -0.0032003 ,  0.01787824,  0.01347096,
        0.0109648 ,  0.04944182,  0.07376195,  0.02802599, -0.03057374,
        0.00824197,  0.00726055, -0.00702859,  0.02722998, -0.04129879,
       -0.01807319, -0.01048668,  0.00406061,  0.0104981 , -0.0066434 ,
        0.01492139,  0.00935778,  0.02110783,  0.00464502,  0.03176497,
       -0.00901751, -0.01605326,  0.00954026, -0.01619722, -0.01252645,
        0.01631749,  0.01242343, -0.0303953 , -0.00758049, -0.00767461,
        0.01613703,  0.02898905,  0.00433689,  0.02949817,  0.00

In [36]:
np.dot(my_vector, men_in_black_vector)

0.7600037

In [37]:
## 'Titanic (1997)'의 벡터
titanic = movie_to_idx[1580]
titanic_vector = als_model.item_factors[titanic]
titanic_vector

array([-0.00656817,  0.0094028 , -0.02596095, -0.00517603,  0.01443409,
        0.00299188, -0.00855858, -0.01299242,  0.00341715, -0.01326967,
        0.03021314,  0.02258575, -0.01205799,  0.02422245, -0.01254962,
       -0.0046878 , -0.02359782,  0.00514713,  0.0003445 , -0.01784759,
        0.01197349,  0.00180007, -0.00139866, -0.02082681, -0.0140435 ,
        0.00346759,  0.01143556,  0.0158788 , -0.01116112, -0.00563349,
        0.01143952,  0.00778315,  0.03324519, -0.01420218,  0.02488708,
        0.04781577,  0.04114428,  0.0046234 , -0.01615397,  0.00962108,
        0.01526745, -0.00254282,  0.04587974,  0.00664275,  0.01356303,
       -0.00484995,  0.01563109,  0.01548687,  0.01259498,  0.0005339 ,
        0.0014317 ,  0.0119743 ,  0.03581763, -0.00204599,  0.04789525,
        0.01591144,  0.00748301,  0.00737652,  0.01952904,  0.01183734,
       -0.00776866, -0.02013474, -0.00648585, -0.00287449,  0.01387788,
        0.02172531, -0.00045063,  0.00021894,  0.01208431, -0.01

In [38]:
np.dot(my_vector, titanic_vector)

0.42970487

In [39]:
## 'Toy Story 2 (1999)' 의 벡터
toy_story_2 = movie_to_idx[1721]
toy_story_2_vector = als_model.item_factors[toy_story_2]
toy_story_2_vector

array([ 0.0447985 ,  0.01126307, -0.03327607,  0.02565593,  0.00880078,
        0.02816045, -0.02351508,  0.00697671,  0.02595522,  0.03044014,
        0.00509895, -0.01549814, -0.00745883, -0.00846637,  0.00038747,
        0.00394617,  0.02278604, -0.0106572 ,  0.01667752,  0.0173351 ,
       -0.0113779 , -0.02412352,  0.00340881,  0.02180759,  0.00142706,
        0.02146064, -0.00329052,  0.01753907,  0.03208223,  0.00350197,
       -0.00928507,  0.00013703,  0.01414691, -0.00916377,  0.00180728,
        0.05044581, -0.00244347,  0.00324625, -0.05655726,  0.02064607,
       -0.02355216,  0.02432589, -0.03075584,  0.0210866 ,  0.01406657,
       -0.01396128,  0.00794374,  0.01293468, -0.00012446,  0.01254342,
        0.03160957,  0.00943429, -0.00702191, -0.00704917,  0.00792232,
       -0.01715801, -0.00188814,  0.01001645,  0.01134555,  0.00543329,
        0.0014602 , -0.00868362,  0.0146127 , -0.03460194, -0.00197425,
       -0.00149967,  0.01715955,  0.02900592,  0.0004603 ,  0.02

In [40]:
np.dot(my_vector, toy_story_2_vector)

0.59489334

'Toy Story (1995)'과 유사한 영화들과의 유사도

In [41]:
movie_id = movie_to_idx[1]
similar_movie = als_model.similar_items(movie_id, N=15)
similar_movie

[(40, 0.99999994),
 (50, 0.7655787),
 (4, 0.52798843),
 (322, 0.5146994),
 (33, 0.49592587),
 (110, 0.47255933),
 (330, 0.43379906),
 (10, 0.3855836),
 (255, 0.37426057),
 (160, 0.3400017),
 (20, 0.33369708),
 (34, 0.32641202),
 (32, 0.32631457),
 (536, 0.31730995),
 (2144, 0.3159945)]

'Toy Story 2 (1999)' 과 유사한 영화들과의 유사도

In [42]:
movie_id = movie_to_idx[3114]
similar_movie = als_model.similar_items(movie_id, N=15)
similar_movie

[(50, 1.0),
 (40, 0.7655787),
 (4, 0.60500145),
 (322, 0.40494907),
 (851, 0.39659852),
 (1706, 0.34935972),
 (32, 0.3353201),
 (33, 0.32044107),
 (277, 0.31568122),
 (474, 0.3051673),
 (16, 0.2992036),
 (1635, 0.29704216),
 (1996, 0.29112864),
 (110, 0.28127414),
 (126, 0.27916795)]

'Men in Black (1997)'   과 유사한 영화들과의 유사도

In [43]:
movie_id = movie_to_idx[1580]
similar_movie = als_model.similar_items(movie_id, N=15)
similar_movie

[(175, 0.9999999),
 (107, 0.806249),
 (92, 0.5925493),
 (62, 0.59000814),
 (150, 0.45334208),
 (145, 0.44330075),
 (124, 0.43046314),
 (82, 0.40646237),
 (375, 0.39272532),
 (138, 0.37827128),
 (3466, 0.37109998),
 (544, 0.34982917),
 (320, 0.34547448),
 (1754, 0.3448766),
 (670, 0.33863208)]

'Titanic (1997)' 과 유사한 영화들과의 유사도

In [44]:
movie_id = movie_to_idx[1721]
similar_movie = als_model.similar_items(movie_id, N=15)
similar_movie

[(27, 0.99999994),
 (384, 0.43714318),
 (385, 0.36285695),
 (3493, 0.36050612),
 (641, 0.35996258),
 (498, 0.35550475),
 (1418, 0.342571),
 (626, 0.33746186),
 (59, 0.32228446),
 (1509, 0.32067144),
 (433, 0.31891754),
 (508, 0.30666268),
 (336, 0.29615644),
 (77, 0.29517177),
 (154, 0.28918162)]

## 7) 내가 좋아하는 영화와 비슷한 영화를 추천받아 봅시다.

In [45]:
def get_similar_movie(favorite_movie):
    movie_id = movie_to_idx[favorite_movie]
    similar_movie = als_model.similar_items(movie_id, N=15)
    idx_to_movie = {v:k for k,v in movie_to_idx.items()}
    recommand_list = [idx_to_movie[i[0]] for i in similar_movie]
    for movie in recommand_list:
        print(movies[movies['movie_id'] == movie], '----------------------------------------------------------------------------------------', sep='\n')

## 'Toy Story 2 (1999)'       

get_similar_movie(3114)

      movie_id               title                        genre
3045      3114  Toy Story 2 (1999)  Animation|Children's|Comedy
----------------------------------------------------------------------------------------
   movie_id             title                        genre
0         1  Toy Story (1995)  Animation|Children's|Comedy
----------------------------------------------------------------------------------------
      movie_id                 title                        genre
2286      2355  Bug's Life, A (1998)  Animation|Children's|Comedy
----------------------------------------------------------------------------------------
    movie_id        title                    genre
33        34  Babe (1995)  Children's|Comedy|Drama
----------------------------------------------------------------------------------------
      movie_id                   title                 genre
2692      2761  Iron Giant, The (1999)  Animation|Children's
------------------------------------------

In [46]:
## 'Men in Black (1997)'  

get_similar_movie(1580)

      movie_id                title                           genre
1539      1580  Men in Black (1997)  Action|Adventure|Comedy|Sci-Fi
----------------------------------------------------------------------------------------
     movie_id                 title                    genre
476       480  Jurassic Park (1993)  Action|Adventure|Sci-Fi
----------------------------------------------------------------------------------------
     movie_id                              title                   genre
585       589  Terminator 2: Judgment Day (1991)  Action|Sci-Fi|Thriller
----------------------------------------------------------------------------------------
      movie_id                title                             genre
2847      2916  Total Recall (1990)  Action|Adventure|Sci-Fi|Thriller
----------------------------------------------------------------------------------------
     movie_id                          title              genre
770       780  Independence Day (ID4

In [47]:
## 'Titanic (1997)'

get_similar_movie(1721)

      movie_id           title          genre
1672      1721  Titanic (1997)  Drama|Romance
----------------------------------------------------------------------------------------
      movie_id                 title          genre
1372      1393  Jerry Maguire (1996)  Drama|Romance
----------------------------------------------------------------------------------------
      movie_id                    title  genre
1636      1682  Truman Show, The (1998)  Drama
----------------------------------------------------------------------------------------
      movie_id           title   genre
3526      3595  Held Up (2000)  Comedy
----------------------------------------------------------------------------------------
    movie_id                      title  genre
61        62  Mr. Holland's Opus (1995)  Drama
----------------------------------------------------------------------------------------
      movie_id                                  title          genre
2056      2125  Ever Aft

In [48]:
## 'Toy Story (1995)' 

get_similar_movie(1)

   movie_id             title                        genre
0         1  Toy Story (1995)  Animation|Children's|Comedy
----------------------------------------------------------------------------------------
      movie_id               title                        genre
3045      3114  Toy Story 2 (1999)  Animation|Children's|Comedy
----------------------------------------------------------------------------------------
      movie_id                 title                        genre
2286      2355  Bug's Life, A (1998)  Animation|Children's|Comedy
----------------------------------------------------------------------------------------
    movie_id        title                    genre
33        34  Babe (1995)  Children's|Comedy|Drama
----------------------------------------------------------------------------------------
     movie_id           title                                genre
584       588  Aladdin (1992)  Animation|Children's|Comedy|Musical
------------------------------

## 8) 내가 가장 좋아할 만한 영화들을 추천받아 봅시다.

In [49]:
def recommand_movie(user_id):
    user = user_to_idx[user_id]
    movie_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
    idx_to_movie = {v:k for k,v in movie_to_idx.items()}
    recommand_list = [idx_to_movie[i[0]] for i in movie_recommended]
    for movie in recommand_list:
        print(movies[movies['movie_id'] == movie], '--------------------------------------------------------------------', sep='\n')

recommand_movie(my_id)

      movie_id                 title                        genre
2286      2355  Bug's Life, A (1998)  Animation|Children's|Comedy
--------------------------------------------------------------------
    movie_id        title                    genre
33        34  Babe (1995)  Children's|Comedy|Drama
--------------------------------------------------------------------
     movie_id                 title                    genre
476       480  Jurassic Park (1993)  Action|Adventure|Sci-Fi
--------------------------------------------------------------------
     movie_id                              title                   genre
585       589  Terminator 2: Judgment Day (1991)  Action|Sci-Fi|Thriller
--------------------------------------------------------------------
      movie_id               title                   genre
2502      2571  Matrix, The (1999)  Action|Sci-Fi|Thriller
--------------------------------------------------------------------
     movie_id                title 

In [57]:
idx_to_movie = {v:k for k,v in movie_to_idx.items()}
[idx_to_movie[i[0]] for i in similar_movie]

[1721,
 1393,
 1682,
 3595,
 62,
 2125,
 1835,
 2424,
 2268,
 1183,
 3259,
 587,
 597,
 2236,
 1784]

In [58]:
## 기여도 체크

def get_contribution(m_id):
    user = user_to_idx[my_id]
    explain = als_model.explain(user, csr_data, itemid=m_id)
    explain = [(idx_to_movie[i[0]], i[1]) for i in explain[1]]
    return explain

get_contribution(2355)  ## 추천받은 첫번째 영화 <2355  Bug's Life, A (1998)>

[(3114, 0.0005615478690651856),
 (1721, 0.0004596653379270268),
 (1, -0.0031226922038830613),
 (1580, -0.006001433584500284)]

# 회고!

## < 어려웠던 점>  
컬럼을 제거하고 필요한 컬럼만 선택해서 처리하는 데이터 전처리 과정이 어려웠던 것 같다. 그리고 영화를 고르는 과정에서 예전 영화들 1900년대 영화들이 많고 2000년대 영화는 없는 경우가 많아서 좋아하는 영화를 정확히 보기는 어려웠다. 그니까 더 좋아하는 영화가 있는데 덜 좋아하는 영화를 어쩔 수 없이 선택을 해서 나중에 영화를 추천해준 것에 정확히 나의 데이터가 안들어가있는 느낌이었다. 그리고 추천된 영화도 모르는 경우가 많아서 영화제목을 찾아보았다. 나중에는 영화 데이터를 좀 더 최신것으로 준비해서 해봐야겠다고 생각이 들었다.

## < 알아낸 점> & < 아직까지 모호한 점>  
- 추천 시스템이란?   
=> 나와 비슷한 다른 사용자들이 좋아하는 것과 비슷한 것을 내게 추쳔해주는 것!   

- 협업 필터링방식과 콘텐츠 기반 필터링방식의 차이점은?  
    - 협업 필터링은 다수의 사용자의 아이템 구매 이력 정보만으로 사용자간 유사성 및 아이템 간 유사성을 파악함  
    - 콘텐츠 기반 필터링은 아이템의 고유의 정보를 바탕으로 아이템 간 유사성을 파악한다.

=> 콘텐츠 기반 필터링에서는 아이템 자체의 속성에만 주목하고 사용자와 아이템 간의 관련성 에는 주목하지 않는다.

추천시스템은  
1. 아이템이 많고,   
2. 유저의 취향은 다양할 때 유저가 소비할 만한 아이템을 예측하는 모델이다.   
ex) 유튜브, 페이스북, 아마존 등이 있다.   

또한, 데이터를 탐색해서 원하는 데이터만 추려서 처리를 해두면, 데이터를 사용하기에 아주 효율적이라는 것을 이번에 깨달았다.

## <시도한 것들>  
시도해본 것은 아니지만! 이 노드를 하면서 이래저래 찾아보면서 데이터를 더 처리할 수 있는 방법들이 많은 것 같다. 특히 나는 영화를 고를 때 제목, 별점도 중요하게 보지만 장르를 먼저 선택하고 고르는 경우가 많다. 그래서 다음에는 데이터를 장르별로 나눠서 정리를 해보고 영화를 골라보는 것도 재밌을 것이라고 생각했다. 그리고 영화 별점이나 평점 순을 그래프나 시각화해서 표현을 해봐도 재밌을 것 같다고 생각을 했다... 생각은 했지만...생각보다 시간이 없어서,, 일단 pass하고 다음에 꼭 해보고 싶다.

## < 자기 다짐>  
으음 뭔가 노드를 하다가 조금 재밌는 노드를 하면 더 하고싶은 욕심이 생길때 가 있는데 그럴 때마다 다 한 노드를 망칠까봐 아니면 값이 틀어질까봐 못했던 경우가 많았는데 제출도 중요하지만 노드에 있는 자료 외에 것도 찾아서 해보는 것도 좋을 것 같다고 생각이 들었다. 위에 적기는 했는데 내가 진짜 좋아하는 영화가 없는 경우에는 그 영화를 별점을 주고 넣으면 어떨까 생각을 해보았다. 예를 들어서 해리포터가 영화 리스트에는 없던데 내가 없는 영화를 별점을 매겨서 저 데이터에 추가를 해주면 어떨까 싶었다. 제발...생각만 하지말고 실천을 해보자~~~